In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import time
import os

In [2]:
page_size = 50
start_page = 10
end_page = 20

In [3]:
endpoint = 'https://www.dailymail.co.uk/home/search.html'
params = {
    'offset': '0',
    'size': str(page_size),
    'sel': 'site',
    'searchPhrase': '',
    'sort': 'recent',
    'channel': 'tvshowbiz',
    'type': 'article',
    'days': 'all'
}

In [4]:
page_content = []
for page in range(start_page, end_page):
    params['offset'] = str(page * page_size)
    new_r = requests.get(endpoint, params=params)
    page_content.append(new_r.text)
    time.sleep(1)

KeyboardInterrupt: 

In [ ]:
end_page * page_size

In [ ]:
page_range = str(start_page * page_size) + '_' +str(end_page * page_size)

In [220]:
page_range

'500_1000'

In [221]:
with open('saved_pages/%s.json' % page_range, 'w') as outfile:
    json.dump(page_content, outfile)

In [9]:
files = os.listdir('saved_pages/')
files.sort()

In [90]:
files = ['200000_250000.json','150000_200000.json']

In [91]:
loaded_html = []
for file in files:
    with open('saved_pages/%s' % file, 'r') as infile:
        d = json.load(infile)
        loaded_html = loaded_html + d

In [92]:
article_bs = []
for html in loaded_html:
    bs = BeautifulSoup(html)
    div_sch_result = bs.find_all('div', class_='sch-result')
    article_bs = article_bs + div_sch_result

In [93]:
headline_list = []
for bs_obj in article_bs:
    h3 = bs_obj.find('h3', class_='sch-res-title')
    h4 = bs_obj.find('h4')
    a = h4.find('a').text if h4.find('a') != None else h4.text
    split = h4.text.split(' -')
    date = split[1] if len(split) > 1 else h4.text

    obj = {
        'headline_text': h3.text,
        'article_url': h3.find('a')['href'],
        'article_preview': bs_obj.find('p', class_='sch-res-preview').text,
        'date': date,
        'author': a
    }
    headline_list.append(obj)

In [94]:
df = pd.DataFrame(headline_list)

In [95]:
df = df.drop_duplicates()

In [96]:
df['date'] = df['date'].str.strip()

In [97]:
def replace(old, new):
    return df['date'].str.replace(old, new)

In [98]:
for i in [['\n', ''], ['th', ''], ['nd',''], ['rd',''], ['st', ''],['Augu ', 'August ']]:
    df['date'] = replace(i[0], i[1])

In [99]:
df['date'] = pd.to_datetime(df['date'], format='%B %d %Y, %I:%M:%S %p')

In [100]:
df = df.sort_values('date')

In [101]:
start = str(df['date'].min())
end = str(df['date'].max())

In [102]:
start, end

('2018-11-13 10:29:15', '2020-02-05 09:38:59')

In [103]:
df.to_csv('saved_headlines/%s to %s.csv' % (start, end), index=False)